# But Why...?

Because it feels ridiculous! Also, if you can find complex patterns in multidimensional data can you find relatively simple patterns from a single input?

# Generating the Data

In [ ]:
import numpy as np

def fizz_buzz_onehot_encoder(n):
    
    if (n % 3) == 0 and (n % 5)==0:
        vector = np.array([0,0,1,0])
    elif n % 3 == 0: 
        vector = np.array([1,0,0,0])
    elif n % 5 == 0:
        vector = np.array([0,1,0,0])
    else:
        vector = np.array([0,0,0,1])
    return vector

def fizz_buzz_encoder(n):
    '''
    Encode any given number to the FizzBuzz representation. That is 'fizz' if
    divisible by 3, 'buzz' if divisible by 5, 'fizzbuzz' if divisible by both
    3 and 5, or simply the number given. The encoded output is a vector 
    representing the output; [1,0] meaning a "fizz", [0,1] a "buzz", and [1,1]
    a "fizzbuzz". Note that the zero vector represents no word (just the 
    number).
    '''
    # Default to nothing in array
    vector = np.array([0,0])
    if n % 3 == 0: 
        vector += np.array([0,1])
    if n % 5 == 0:
        vector += np.array([1,0])
        
    return vector

def fizz_buzz_decoder(n_vector, n_default=''):
    '''
    Decodes the fizzbuzz vector representation to an output. See 
    fizz_buzz_encoder for details on the representation. n_default is the 
    default string representation if 'fizz', 'buzz' or 'fizzbuzz' should not
    be used in the string representation (usually just the number)
    '''
    # Use the vector to create the different 
    output = n_vector[0] * 'fizz' + n_vector[1] * 'buzz'
    # If zero vector, it should printout the default number
    if n_vector.sum() == 0:
        output = str(n_default)
    return output

def binary_encoder(n, binary_digits=10):
    '''
    Creates a binary representation array from a given integer
    
    Args:
    n             Integer to encode into a binary array
    binary_digits (default 10) Number of digits in binary representation 
                  (leading 0s)
                  
    Return:
    NumPy array of binary representation of integer
    '''
    # Use the binary iterator for efficiency
    bin_iter = (n >> d & 1 for d in range(binary_digits))
    return np.fromiter(bin_iter, int)

def decimal_encoder(n, decimal_digits=4):
    '''
    Creates a decimal representation array from a given integer
    
    Args:
    n              Integer to encode into a decimal array
    decimal_digits (default 4) Number of digits in decimal representation 
                   (leading 0s)
                  
    Return:
    NumPy array of binary representation of integer
    '''
    # Use the decimal iterator for efficiency
    dec_iter = (n % (10**(d+1)) // (10**d) for d in range(decimal_digits))
    return np.fromiter(dec_iter, int)

In [ ]:
n_start = 101
n_end = 1000

In [ ]:
# Binary representation of number
X = np.array([binary_encoder(i,10) for i in range(n_start, n_end)])
# Get classess/output
Y = np.array([fizz_buzz_onehot_encoder(i) for i in range(n_start, n_end)])

## Split into train-valid

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(
    X, Y, test_size=0.2, random_state=27)

# Modeling

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import models, layers, optimizers
from keras.callbacks import ModelCheckpoint  
from keras import losses
import os 

## A Simple Model

In [ ]:
model_name = 'bin_simple_hl'

model = Sequential()
# Attempt to embed whether or not it is divisible by 3 or 5 or neither
model.add(Dense(units=3, activation='relu', input_dim=X.shape[1], name='input_layer'))
# Classification
model.add(Dense(units=4, activation='softmax'))

model.summary()

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy, 
              metrics=['accuracy'],
              optimizer=optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True))

### Fit the model

In [ ]:
from keras.callbacks import ModelCheckpoint  
import os 

epochs = 1600
batch_size = 16

# Create a saved models directory
model_dir = 'models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
model_path = f'{model_dir}/weights.best.{model_name}.hdf5'
    
checkpointer = ModelCheckpoint(filepath=model_path, 
                               verbose=1, save_best_only=True)

model.fit(x_train, y_train,
          validation_data=(x_valid, y_valid),
          epochs=epochs, batch_size=batch_size, 
          callbacks=[checkpointer], verbose=2, shuffle=True)

### Evaluation

In [ ]:
# TODO

In [ ]:
classes = model.predict(x_valid)
predictions = np.argmax(classes, axis=1) == np.argmax(y_valid, axis=1)

predictions.sum() / predictions.size

In [ ]:
n_start, n_end = 1, 101
x_test = np.array([binary_encoder(i,10) for i in range(n_start, n_end)])
y_test = np.array([fizz_buzz_onehot_encoder(i) for i in range(n_start, n_end)])

classes = model.predict(x_test)
predictions = np.argmax(classes, axis=1) == np.argmax(y_test, axis=1)

predictions.sum() / predictions.size


In [ ]:
classes[predictions == False]
for v in x_test[predictions == False]:
    t = 0
    for i in range(len(v)):
        t += v[i] * 2 ** i
    print(t)

## More Complexity Model

In [ ]:
model_name = 'bin_3_hl_3_dropout'

model = Sequential()
# 
model.add(Dense(units=128, activation='relu', input_dim=X.shape[1], name='input_layer'))
model.add(Dropout(0.2))
model.add(Dense(units=128, activation='relu', name='hl_0'))
model.add(Dropout(0.2))
model.add(Dense(units=128, activation='relu', name='hl_1'))
model.add(Dropout(0.2))
# Classification
model.add(Dense(units=4, activation='softmax'))

model.summary()

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy, 
              metrics=['accuracy'],
              optimizer=optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True))

### Fit the model

In [ ]:
from keras.callbacks import ModelCheckpoint  
import os 

epochs = 3200
batch_size = 32

# Create a saved models directory
model_dir = 'models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
model_path = f'{model_dir}/weights.best.{model_name}.hdf5'
    
checkpointer = ModelCheckpoint(filepath=model_path, 
                               verbose=1, save_best_only=True)

model.fit(x_train, y_train,
          validation_data=(x_valid, y_valid),
          epochs=epochs, batch_size=batch_size, 
          callbacks=[checkpointer], verbose=2, shuffle=True)

### Evaluation

In [ ]:
# TODO

In [ ]:
classes = model.predict(x_valid)
predictions = np.argmax(classes, axis=1) == np.argmax(y_valid, axis=1)

predictions.sum() / predictions.size

In [ ]:
n_start, n_end = 1, 101
x_test = np.array([binary_encoder(i,10) for i in range(n_start, n_end)])
y_test = np.array([fizz_buzz_onehot_encoder(i) for i in range(n_start, n_end)])

classes = model.predict(x_test)
predictions = np.argmax(classes, axis=1) == np.argmax(y_test, axis=1)

predictions.sum() / predictions.size


In [ ]:
classes[predictions == False]
for v in x_test[predictions == False]:
    t = 0
    for i in range(len(v)):
        t += v[i] * 2 ** i
    print(t)